In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Definimos las transformaciones para preprocesado de las imágenes
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Normalización con media y desviación estándar de MNIST
])

# Cargamos el dataset MNIST
train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)

# Creamos los dataloaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

from torch.nn import functional as F

class ImprovedCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(3 * 3 * 128, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout(x)
        x = x.view(-1, 3 * 3 * 128)  # Aplanamiento tras las capas convolucionales
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)  # Use log_softmax for cross-entropy loss

model = ImprovedCNN()  # Instanciamos la red neuronal
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.train()
num_epochs = 10

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

model.eval()  # Ponemos el modelo en modo evaluación

correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network on the 10000 test images: {accuracy:.2f}%')


Epoch [1/10], Step [100/938], Loss: 0.2587
Epoch [1/10], Step [200/938], Loss: 0.1111
Epoch [1/10], Step [300/938], Loss: 0.1187
Epoch [1/10], Step [400/938], Loss: 0.0913
Epoch [1/10], Step [500/938], Loss: 0.1477
Epoch [1/10], Step [600/938], Loss: 0.3752
Epoch [1/10], Step [700/938], Loss: 0.0897
Epoch [1/10], Step [800/938], Loss: 0.0551
Epoch [1/10], Step [900/938], Loss: 0.0789
Epoch [2/10], Step [100/938], Loss: 0.0990
Epoch [2/10], Step [200/938], Loss: 0.0510
Epoch [2/10], Step [300/938], Loss: 0.0222
Epoch [2/10], Step [400/938], Loss: 0.0880
Epoch [2/10], Step [500/938], Loss: 0.0276
Epoch [2/10], Step [600/938], Loss: 0.0390
Epoch [2/10], Step [700/938], Loss: 0.0357
Epoch [2/10], Step [800/938], Loss: 0.0080
Epoch [2/10], Step [900/938], Loss: 0.0468
Epoch [3/10], Step [100/938], Loss: 0.0056
Epoch [3/10], Step [200/938], Loss: 0.0122
Epoch [3/10], Step [300/938], Loss: 0.0444
Epoch [3/10], Step [400/938], Loss: 0.0679
Epoch [3/10], Step [500/938], Loss: 0.0782
Epoch [3/10